In [3]:
from tqdm import tqdm
import csv

with open("gec_val_data.csv", "w") as f:
    write = csv.writer(f, escapechar="\\")
    write.writerow(["in", "out"])
    for i in tqdm(range(10)):
        with open(f"./gec_data/C4_200M.tsv-0000{i}-of-00010") as d:
            inp_out_pairs = d.readlines()
            d_data = [list(map(str.strip, inp_out_pairs[j].split("\t"))) for j in tqdm(range(10_000, 10_100))]
            write.writerows(d_data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:05<00:00, 24.56s/it]


In [21]:
import pandas as pd
val_df = pd.read_csv('gec_val_data.csv')
val_df.head()

from transformers import AutoModelForSeq2SeqLM, T5Tokenizer
gec_tokenizer = T5Tokenizer.from_pretrained("./gec_model_final")
gec_model = AutoModelForSeq2SeqLM.from_pretrained("./gec_model_final")

In [22]:
from tqdm import tqdm
from datasets import load_metric
rouge_metric = load_metric("rouge")

BATCH_SIZE = 10
corrections = []

for i in tqdm(range(0, 1000, BATCH_SIZE)):
    batch = gec_tokenizer(list(val_df["in"])[i:i + BATCH_SIZE], padding='max_length', max_length=512, return_tensors="pt")
    translated = gec_model.generate(**batch,num_beams=5, num_return_sequences=1)
    corrs = gec_tokenizer.batch_decode(translated, padding="longest", skip_special_tokens=True)
    corrections.extend(corrs)

rouge_data = rouge_metric.compute(predictions=corrections, references=list(val_df["out"]), use_stemmer=True)
for key, val in rouge_data.items():
    print(key)
    print(val)

  0%|                                                                                                           | 0/100 [00:00<?, ?it/s]/Users/archit/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [16:16<00:00,  9.76s/it]


rouge1
AggregateScore(low=Score(precision=0.8826401603984881, recall=0.6130604962748207, fmeasure=0.6906303879704263), mid=Score(precision=0.890337569237607, recall=0.6288194815875836, fmeasure=0.7034409218696728), high=Score(precision=0.8980534560034906, recall=0.6437013821902137, fmeasure=0.7150787168742926))
rouge2
AggregateScore(low=Score(precision=0.761314086101807, recall=0.516606105273693, fmeasure=0.5851903975072961), mid=Score(precision=0.7742637407119028, recall=0.5315795089711381, fmeasure=0.5988546119950859), high=Score(precision=0.7877910777212989, recall=0.5484305309330766, fmeasure=0.6139823988448598))
rougeL
AggregateScore(low=Score(precision=0.8715967092463133, recall=0.6069260159978522, fmeasure=0.6836461725057109), mid=Score(precision=0.8800923771339606, recall=0.6222463640100695, fmeasure=0.6960447067503552), high=Score(precision=0.8890362842771916, recall=0.6382578912320958, fmeasure=0.7093919798495942))
rougeLsum
AggregateScore(low=Score(precision=0.87123865520637

In [23]:
bleu_metric = load_metric("sacrebleu")
bleu_data = bleu_metric.compute(predictions=[corrections], references=[list(val_df["out"])])
for key, val in bleu_data.items():
    print(key)
    print(val)

score
38.30013296158841
counts
[4592, 7395, 6339, 5029]
totals
[14978, 14977, 14976, 14975]
precisions
[30.65829883829617, 49.375709421112376, 42.32772435897436, 33.58263772954925]
bp
1.0
sys_len
14978
ref_len
167


In [29]:
!pip install jiwer
wer_metric = load_metric("wer")
wer_score = wer_metric.compute(predictions=corrections, references=list(val_df["out"]))
print(wer_score)

0.580658159838818


In [31]:
perplexity_metric = load_metric("perplexity")
perplexity_score = perplexity_metric.compute(input_texts=corrections, model_id='gpt2')
print(perplexity_score)

Using pad_token, but it is not set yet.


  0%|          | 0/63 [00:00<?, ?it/s]

{'perplexities': [34.37554168701172, 294.5758972167969, 14.330902099609375, 56.59889221191406, 118.72037506103516, 61.264434814453125, 42.92140579223633, 7.470332145690918, 12.173117637634277, 14.392556190490723, 12.475842475891113, 36.83244323730469, 18.53238296508789, 51.04104232788086, 21.754207611083984, 213.0064239501953, 47.9545783996582, 31.314729690551758, 63.97848892211914, 17.82229232788086, 53.85265350341797, 49.69066619873047, 36.100341796875, 17.686323165893555, 91.73695373535156, 192.50827026367188, 99.80016326904297, 99.16048431396484, 21.53976058959961, 202.28672790527344, 80.42808532714844, 25.599130630493164, 98.87709045410156, 29.80580711364746, 43.8544921875, 26.47821807861328, 7.085689544677734, 13.159878730773926, 194.55540466308594, 46.758827209472656, 129.05398559570312, 55.352237701416016, 83.45198822021484, 44.46369552612305, 64.15731811523438, 31.10428810119629, 238.88519287109375, 51.779624938964844, 72.11502075195312, 121.54276275634766, 305.1803283691406, 